<a href="https://colab.research.google.com/github/AnjuAravind12/Web_Scraping_call-transcripts/blob/main/seekingalphascrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
from bs4 import BeautifulSoup
import re
from collections import Counter
import time
import csv
import pandas as pd
import urllib.request  as urllib2 
#from urllib2 import HTTPError

In [2]:
from urllib.request import HTTPError

In [16]:
import requests
import time
from bs4 import BeautifulSoup

In [17]:
def get_date(c):
    end = c.find('|')
    return c[0:end-1]

In [18]:
def get_ticker(c):
    beg = c.find('(')
    end = c.find(')')
    return c[beg+1:end]

def grab_page(url):
    print("attempting to grab page: " + url)
    page = requests.get(url)
    page_html = page.text
    soup = BeautifulSoup(page_html, 'html.parser')

    meta = soup.find("div",{'class':'a-info get-alerts'})
    content = soup.find(id="a-body")

    if type(meta) or type(content) == "NoneType":
        print("skipping this link, no content here")
        return
    else:
        text = content.text
        mtext = meta.text

        filename = get_ticker(mtext) + "_" + get_date(mtext)
        file = open(filename.lower() + ".txt", 'w')
        file.write(text)
        file.close
        print(filename.lower()+ " sucessfully saved")


In [19]:
def process_list_page(i):
    origin_page = "https://seekingalpha.com/earnings/earnings-call-transcripts" + "/" + str(i)
    print("getting page " + origin_page)
    page = requests.get(origin_page)
    page_html = page.text
    #print(page_html)
    soup = BeautifulSoup(page_html, 'html.parser')
    alist = soup.find_all("li",{'class':'list-group-item article'})
    for i in range(0,len(alist)):
        url_ending = alist[i].find_all("a")[0].attrs['href']
        url = "https://seekingalpha.com" + url_ending
        grab_page(url)
        time.sleep(.5)

In [20]:
for i in range(1,200): #choose what pages of earnings to scrape
    process_list_page(i)


getting page https://seekingalpha.com/earnings/earnings-call-transcripts/1
getting page https://seekingalpha.com/earnings/earnings-call-transcripts/2
attempting to grab page: https://seekingalpha.com/article/4391381-greencore-group-plc-gncgf-ceo-patrick-coveney-on-q4-2020-results-earnings-call-transcript
skipping this link, no content here
attempting to grab page: https://seekingalpha.com/article/4391379-haynes-international-inc-hayn-ceo-mike-shor-on-q4-2020-results-earnings-call-transcript
skipping this link, no content here
attempting to grab page: https://seekingalpha.com/article/4391376-jacobs-engineering-group-inc-j-ceo-steve-demetriou-on-q4-2020-results-earnings-call
skipping this link, no content here
attempting to grab page: https://seekingalpha.com/article/4391374-hormel-foods-corporation-hrl-ceo-jim-snee-on-q4-2020-results-earnings-call-transcript
skipping this link, no content here
attempting to grab page: https://seekingalpha.com/article/4391369-monmouth-real-estate-investm

In [44]:
link = "https://seekingalpha.com/earnings/earnings-call-transcripts/191"


In [45]:
def get_data(link):
    for url in [link.format(page) for page in range(1,6)]:
        res = requests.get(url)
        soup = BeautifulSoup(res.text,"lxml")

        for items in soup.select("table.table tr"):
            td = [item.get_text(strip=True) for item in items.select("th,td")]
            writer.writerow(td)

In [46]:
if __name__ == '__main__':
    with open("tabularitem.csv","w",newline="") as infile: #if encoding issue comes up then replace with ('tabularitem.csv', 'w', newline="", encoding="utf-8")
        writer = csv.writer(infile)
        get_data(link)

In [37]:
filename = "calltranscripts.csv"
csv_writer = csv.writer(open(filename, 'w'))


In [38]:
for tr in soup.find_all("tr"):
    data = []
    # for headers ( entered only once - the first time - )
    for th in tr.find_all("th"):
        data.append(th.text)
    if data:
        print("Inserting headers : {}".format(','.join(data)))
        csv_writer.writerow(data)
        continue

    for td in tr.find_all("td"):
        if td.a:
            data.append(td.a.text.strip())
        else:
            data.append(td.text.strip())
    if data:
        print("Inserting data: {}".format(','.join(data)))
        csv_writer.writerow(data)